In [25]:
# Import required libraries
import numpy as np, pandas as pd
from scipy import stats
from h3 import h3
from scipy import stats
import json
from shapely.ops import unary_union
from shapely.geometry import shape, Point, Polygon, MultiPolygon, LineString
import geopandas as gpd
from geopandas import GeoDataFrame
import warnings
import hashlib
import pickle
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.svm import SVR
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import itertools


# Filter out annoying warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [36]:
!wget https://www.duscloud.eu/s/pEGQo8yTKMpDd3G/download -O taxi_trips_with_weather_cleaned.csv

--2024-06-28 13:16:56--  https://www.duscloud.eu/s/pEGQo8yTKMpDd3G/download
Auflösen des Hostnamens www.duscloud.eu (www.duscloud.eu)… 2a02:8071:5351:a2a0:da3a:ddff:fed7:199b, 84.118.33.125
Verbindungsaufbau zu www.duscloud.eu (www.duscloud.eu)|2a02:8071:5351:a2a0:da3a:ddff:fed7:199b|:443 … fehlgeschlagen: Operation timed out.
Verbindungsaufbau zu www.duscloud.eu (www.duscloud.eu)|84.118.33.125|:443 … verbunden.
HTTP-Anforderung gesendet, auf Antwort wird gewartet … 200 OK
Länge: 1046378872 (998M) [text/csv]
Wird in »taxi_trips_with_weather_cleaned.csv« gespeichert.

taxi_trips_with_wea 100%[===================>] 997,90M  5,93MB/s    in 2m 51s  

2024-06-28 13:21:03 (5,83 MB/s) - »taxi_trips_with_weather_cleaned.csv« gespeichert [1046378872/1046378872]



In [37]:
ls 

Boundaries - Community Areas.geojson  df_poi.csv
Duracell_AAA_2024-master/             ds/
SVR.ipynb                             taxi_trips_with_weather_cleaned.csv


In [69]:
dataset = pd.read_csv("taxi_trips_with_weather_cleaned.csv")
dataset.head()

,trip_id,taxi_id,timestamp_start,timestamp_end,duration,distance,fare,tips,tolls,extras,...,dropoff_long,dropoff_location,temperature,dew_point,humidity,wind_speed,wind_gust,pressure,precipitation_rate,precipitation_accumulation
0,0cd46aa628ea5f871630113eb8ba92e4,09a6eaba341c43fc653756deff43a50f,2020-01-01 00:00:00,2020-01-01 00:15:00,180.0,0.64,4.50,0.00,0.0,1.0,...,-87.626211,POINT (-87.6262105324 41.8991556134),-2.83,-5.33,83.0,0.48,0.64,1.0088,0.0,0.0
1,4fbe84436a116f11c51ef3306e7adf0c,0f66b306ebea0f05dd124bf958729b64,2020-01-01 00:00:00,2020-01-01 00:30:00,1723.0,1.13,12.00,0.00,0.0,0.0,...,-87.626215,POINT (-87.6262149064 41.8925077809),-2.83,-5.33,83.0,0.48,0.64,1.0088,0.0,0.0
2,08a197de2703972bad8877628b18bb30,5a3410255ea0041b967e74090838b14d,2020-01-01 00:00:00,2020-01-01 00:00:00,420.0,0.97,6.00,0.00,0.0,1.0,...,-87.618868,POINT (-87.6188683546 41.8909220259),-2.83,-5.33,83.0,0.48,0.64,1.0088,0.0,0.0
3,94f8c8a4624a061988e0f32e12be04c5,3049f212da83687eb7ea5cb9d3046d89,2020-01-01 00:00:00,2020-01-01 00:30:00,1320.0,8.21,16.75,3.85,0.0,2.0,...,-87.676356,POINT (-87.6763559892 41.9012069941),-2.83,-5.33,83.0,0.48,0.64,1.0088,0.0,0.0
4,dd25d5229b1ce93ea03b7c9357e6f4f9,d9d9d16f34141de38105a060abd8a90c,2020-01-01 00:00:00,2020-01-01 00:15:00,504.0,1.19,6.25,0.00,0.0,1.0,...,-87.626215,POINT (-87.6262149064 41.8925077809),-2.83,-5.33,83.0,0.48,0.64,1.0088,0.0,0.0


In [39]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3129456 entries, 0 to 3129455
Data columns (total 27 columns):
 #   Column                      Dtype  
---  ------                      -----  
 0   trip_id                     object 
 1   taxi_id                     object 
 2   timestamp_start             object 
 3   timestamp_end               object 
 4   duration                    float64
 5   distance                    float64
 6   fare                        float64
 7   tips                        float64
 8   tolls                       float64
 9   extras                      float64
 10  total                       float64
 11  payment_type                object 
 12  company                     object 
 13  pickup_lat                  float64
 14  pickup_long                 float64
 15  pickup_location             object 
 16  dropoff_lat                 float64
 17  dropoff_long                float64
 18  dropoff_location            object 
 19  temperature          

In [70]:
columns_to_keep = [
    "timestamp_start", "pickup_lat", "pickup_long",
    "temperature", "humidity", "wind_speed", "wind_gust", 
    "precipitation_rate", "precipitation_accumulation"
]

# Copy the dataset and keep only the specified columns
d_c = dataset.copy()
d_c = d_c.loc[:, columns_to_keep]

# Display the first few rows of the new DataFrame to verify the changes
print(d_c.head())

       timestamp_start  pickup_lat  pickup_long  temperature  humidity  \
0  2020-01-01 00:00:00   41.898332   -87.620763        -2.83      83.0   
1  2020-01-01 00:00:00   41.892042   -87.631864        -2.83      83.0   
2  2020-01-01 00:00:00   41.895033   -87.619711        -2.83      83.0   
3  2020-01-01 00:00:00   41.965812   -87.655879        -2.83      83.0   
4  2020-01-01 00:00:00   41.898332   -87.620763        -2.83      83.0   

   wind_speed  wind_gust  precipitation_rate  precipitation_accumulation  
0        0.48       0.64                 0.0                         0.0  
1        0.48       0.64                 0.0                         0.0  
2        0.48       0.64                 0.0                         0.0  
3        0.48       0.64                 0.0                         0.0  
4        0.48       0.64                 0.0                         0.0  


In [71]:
# Convert pickup and dropoff locations to hexagons
d_c['pickup_hex'] = d_c.apply(lambda row: h3.geo_to_h3(row['pickup_lat'], row['pickup_long'], resolution=8), axis=1)
#d_c['dropoff_hex'] = d_c.apply(lambda row: h3.geo_to_h3(row['dropoff_lat'], row['dropoff_long'], resolution=8), axis=1)

# Convert timestamp columns to datetime
d_c['timestamp_start'] = pd.to_datetime(d_c['timestamp_start'])
#d_c['timestamp_end'] = pd.to_datetime(d_c['timestamp_end'])

# Extract time features
d_c['hour_of_day'] = d_c['timestamp_start'].dt.hour
d_c['day_of_week'] = d_c['timestamp_start'].dt.dayofweek
d_c['date'] = d_c['timestamp_start'].dt.date

d_c.drop(columns=['timestamp_start', 'pickup_lat', 'pickup_long'], inplace=True)

d_c.head()

,temperature,humidity,wind_speed,wind_gust,precipitation_rate,precipitation_accumulation,pickup_hex,hour_of_day,day_of_week,date
0,-2.83,83.0,0.48,0.64,0.0,0.0,882664c1e9fffff,0,2,2020-01-01
1,-2.83,83.0,0.48,0.64,0.0,0.0,882664c1e7fffff,0,2,2020-01-01
2,-2.83,83.0,0.48,0.64,0.0,0.0,882664c1ebfffff,0,2,2020-01-01
3,-2.83,83.0,0.48,0.64,0.0,0.0,882664d897fffff,0,2,2020-01-01
4,-2.83,83.0,0.48,0.64,0.0,0.0,882664c1e9fffff,0,2,2020-01-01


In [74]:
#testen ob wirklich keine Werte da existieren... siehe unten 
filter_date = pd.to_datetime("2020-01-01").date()
#filter_hour =pd.to_datetime(23).hour()
de = d_c[(d_c["pickup_hex"]== "882664c1e9fffff") & (d_c["date"]== filter_date) & (d_c["hour_of_day"] == "23" )]
de

,temperature,humidity,wind_speed,wind_gust,precipitation_rate,precipitation_accumulation,pickup_hex,hour_of_day,day_of_week,date


In [75]:
# Calculate demand per hexagon per hour per date
taxi_demand = d_c.groupby(['pickup_hex', 'date', 'hour_of_day', 'day_of_week']).size().reset_index(name='demand')

# Merge the demand back into the original DataFrame, dropping duplicates
d_c = d_c.drop_duplicates(subset=['pickup_hex', 'date', 'hour_of_day', 'day_of_week'])
d_c = d_c.merge(taxi_demand, on=['pickup_hex', 'date', 'hour_of_day', 'day_of_week'], how='left')

d_c

,temperature,humidity,wind_speed,wind_gust,precipitation_rate,precipitation_accumulation,pickup_hex,hour_of_day,day_of_week,date,demand
0,-2.83,83.0,0.48,0.64,0.0,0.0,882664c1e9fffff,0,2,2020-01-01,37
1,-2.83,83.0,0.48,0.64,0.0,0.0,882664c1e7fffff,0,2,2020-01-01,88
2,-2.83,83.0,0.48,0.64,0.0,0.0,882664c1ebfffff,0,2,2020-01-01,16
3,-2.83,83.0,0.48,0.64,0.0,0.0,882664d897fffff,0,2,2020-01-01,14
4,-2.83,83.0,0.48,0.64,0.0,0.0,882664520bfffff,0,2,2020-01-01,6
...,...,...,...,...,...,...,...,...,...,...,...
340869,-2.94,71.0,0.16,0.16,0.0,0.0,882664cc4dfffff,23,3,2020-12-31,2
340870,-2.94,71.0,0.16,0.16,0.0,0.0,882664cce7fffff,23,3,2020-12-31,1
340871,-2.94,71.0,0.16,0.16,0.0,0.0,882664c80bfffff,23,3,2020-12-31,1
340872,-2.94,71.0,0.16,0.16,0.0,0.0,882664cf55fffff,23,3,2020-12-31,1


In [76]:
## Look at missing values, keine Werte für gewisse Stunden (siehe oben)
filter_date = pd.to_datetime("2020-01-01").date()
de = d_c[(d_c["pickup_hex"]== "882664c1e9fffff") & (d_c["date"]== filter_date)]
de.head()

,temperature,humidity,wind_speed,wind_gust,precipitation_rate,precipitation_accumulation,pickup_hex,hour_of_day,day_of_week,date,demand
0,-2.83,83.0,0.48,0.64,0.0,0.0,882664c1e9fffff,0,2,2020-01-01,37
62,-3.44,84.0,0.16,0.32,0.0,0.0,882664c1e9fffff,1,2,2020-01-01,54
139,-3.94,85.0,0.00,0.00,0.0,0.0,882664c1e9fffff,2,2,2020-01-01,26
241,-4.17,85.0,0.00,0.00,0.0,0.0,882664c1e9fffff,3,2,2020-01-01,5
402,-4.83,84.0,0.16,0.16,0.0,0.0,882664c1e9fffff,7,2,2020-01-01,2


## ADD POI

In [78]:
df_poi = pd.read_csv("df_poi.csv")
df_poi

,geometry,community,NO_AMENITY,bar,cafe,cinema,clinic,library,restaurant,school,theatre,university
0,POLYGON ((-87.73940951036303 41.68655709853868...,MOUNT GREENWOOD,1,0,0,0,0,0,0,0,0,0
1,POLYGON ((-87.73520826155185 41.68718308268451...,MOUNT GREENWOOD,1,0,0,0,0,0,0,0,0,0
2,POLYGON ((-87.73818834568421 41.68946673581943...,MOUNT GREENWOOD,1,0,0,0,0,0,0,0,0,0
3,POLYGON ((-87.73398680338163 41.69009271380396...,MOUNT GREENWOOD,1,0,0,0,0,0,0,0,0,0
4,POLYGON ((-87.73696702370944 41.69237647172979...,MOUNT GREENWOOD,1,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
5624,POLYGON ((-87.53181734817565 41.64640551579936...,HEGEWISCH,1,0,0,0,0,0,0,0,0,0
5625,POLYGON ((-87.53058357592732 41.64931221337571...,HEGEWISCH,1,0,0,0,0,0,0,0,0,0
5626,"POLYGON ((-87.5293496450476 41.65221900915927,...",HEGEWISCH,1,0,0,0,0,0,0,0,0,0
5627,"POLYGON ((-87.52638073045128 41.6499302774579,...",HEGEWISCH,1,0,0,0,0,0,0,0,0,0


In [80]:
#df_poi_l['geometry'] = df_poi_l['geometry'].apply(lambda x: Polygon([tuple(map(float, coord.split())) for coord in x.replace('POLYGON ((', '').replace('))', '').split(', ')]))
#df_poi['hex_id'] = df_poi_l['geometry'].apply(lambda x: h3.geo_to_h3(x.centroid.y, x.centroid.x, resolution=8))
def polygon_to_hex(polygon_str, resolution=8):
    # Convert the POLYGON string to a list of tuples (coordinates)
    coords = [tuple(map(float, coord.split())) for coord in polygon_str.replace('POLYGON ((', '').replace('))', '').split(', ')]
    polygon = Polygon(coords)
    # Get the centroid of the polygon
    centroid = polygon.centroid
    # Convert the centroid to an H3 hexagon ID
    hex_id = h3.geo_to_h3(centroid.y, centroid.x, resolution)
    return hex_id

# Apply the function to the 'geometry' column
df_poi['hex_id'] = df_poi['geometry'].apply(polygon_to_hex)

df_poi.head()

,geometry,community,NO_AMENITY,bar,cafe,cinema,clinic,library,restaurant,school,theatre,university,hex_id
0,POLYGON ((-87.73940951036303 41.68655709853868...,MOUNT GREENWOOD,1,0,0,0,0,0,0,0,0,0,8826645721fffff
1,POLYGON ((-87.73520826155185 41.68718308268451...,MOUNT GREENWOOD,1,0,0,0,0,0,0,0,0,0,882664572bfffff
2,POLYGON ((-87.73818834568421 41.68946673581943...,MOUNT GREENWOOD,1,0,0,0,0,0,0,0,0,0,8826645721fffff
3,POLYGON ((-87.73398680338163 41.69009271380396...,MOUNT GREENWOOD,1,0,0,0,0,0,0,0,0,0,882664572bfffff
4,POLYGON ((-87.73696702370944 41.69237647172979...,MOUNT GREENWOOD,1,0,0,0,0,0,0,0,0,0,8826645729fffff


In [97]:
merged_df = pd.merge(df_poi, d_c, how='right', left_on='hex_id', right_on='pickup_hex')

merged_df_tr = merged_df.drop(columns=["geometry","community"])
merged_df_tr
merged_df_tr.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2354800 entries, 0 to 2354799
Data columns (total 22 columns):
 #   Column                      Dtype  
---  ------                      -----  
 0   NO_AMENITY                  int64  
 1   bar                         int64  
 2   cafe                        int64  
 3   cinema                      int64  
 4   clinic                      int64  
 5   library                     int64  
 6   restaurant                  int64  
 7   school                      int64  
 8   theatre                     int64  
 9   university                  int64  
 10  hex_id                      object 
 11  temperature                 float64
 12  humidity                    float64
 13  wind_speed                  float64
 14  wind_gust                   float64
 15  precipitation_rate          float64
 16  precipitation_accumulation  float64
 17  pickup_hex                  object 
 18  hour_of_day                 int32  
 19  day_of_week          

In [91]:
## Training

In [98]:
X = merged_df_tr.drop(columns=['demand'])
y = merged_df_tr['demand']

In [99]:
def trainsvmmodel(X, y, maxiter=1000, splitratio=0.2, kernel='linear'):

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=splitratio)

    #scaling input data
    #scaler = StandardScaler()
    #X_train_scaled = scaler.fit_transform(X_train)
    #X_test_scaled = scaler.transform(X_test)

    svm_reg = make_pipeline(StandardScaler(), SVR(kernel=kernel, max_iter = maxiter))

    #part 2: applying grid search to find best model parameters
    #if kernel=='linear':
    #    param_grid = {'C': [0.1, 1, 10, 100, 1000], 'epsilon': [0.0001, 0.001, 0.1, 0.2, 0.5, 0.9]}
    #else:
    #    param_grid = {'C': [0.1, 1, 10, 100, 1000], 'gamma':('auto', 'scale'), 'epsilon': [0.0001, 0.001, 0.1, 0.2, 0.5, 0.9]}
    if kernel == 'linear':
        param_grid = {
            'svr__C': [0.1, 1, 10, 100, 1000],
            'svr__epsilon': [0.0001, 0.001, 0.1, 0.2, 0.5, 0.9]
        }
    else:
        param_grid = {
            'svr__C': [0.1, 1, 10, 100, 1000],
            'svr__gamma': ['auto', 'scale'],
            'svr__epsilon': [0.0001, 0.001, 0.1, 0.2, 0.5, 0.9]

        }

    grid_search = GridSearchCV(svm_reg, param_grid, cv=5)
    grid_search.fit(X_train, y_train) ## error

    best_params =grid_search.best_params_

    return best_params, grid_search.best_estimator_, grid_search.best_score_

In [100]:
best_params, best_model, best_score = trainsvmmodel(X, y)
print("Best Parameters:", best_params)
print("Best Score:", best_score)

ValueError: 
All the 150 fits failed.
It is very likely that your model is misconfigured.
You can try to debug the error by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
30 fits failed with the following error:
Traceback (most recent call last):
  File "/usr/local/lib/python3.9/site-packages/sklearn/model_selection/_validation.py", line 888, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/usr/local/lib/python3.9/site-packages/sklearn/base.py", line 1473, in wrapper
    return fit_method(estimator, *args, **kwargs)
  File "/usr/local/lib/python3.9/site-packages/sklearn/pipeline.py", line 472, in fit
    Xt = self._fit(X, y, routed_params)
  File "/usr/local/lib/python3.9/site-packages/sklearn/pipeline.py", line 409, in _fit
    X, fitted_transformer = fit_transform_one_cached(
  File "/usr/local/lib/python3.9/site-packages/joblib/memory.py", line 312, 